<a href="https://colab.research.google.com/github/sayandas96476/RAG/blob/main/Multimodal_RAG_ImageChat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Workflow:
Text extracted from image using OCR

Retriever: CONTEXT is generated using Dense Retrieval using E5 Large Model

Generator: Prompt is used(Model: mixtral-8x7b-32768)


In [2]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from PIL import Image
image = Image.open('/content/drive/MyDrive/testimage.jpg')


In [4]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (3,245 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [5]:
import pytesseract

def extract_text(image):

    text = pytesseract.image_to_string(image)
    return text
ans = extract_text(image)

In [6]:
print(ans)

9:30 B neal

% timesofindia.indiatimes.com/india, e A

 

— THE TIMES OF INDIA

NEWS INDIA TOIGAMES VIDEOS WORLD TOI

Bim 151) Live Score Maha Kumbh 2025 Actor Sa

India taking part in nine-nation naval
exercise between Indian & Pacific
Oceans

Rajat Pandit / TNN / Jan 18, 2025, 18:55 IST

 

India, along with eight other nations, participates in
the French-hosted ‘La Perouse’ naval exercise near
the Malacca, Sunda and Lombok straits to enhance

[] FOLLOW US

NEW DELHI: India and eight other nations are

now taking part in a major naval exercise hosted

   



In [7]:
!pip install -q langchain_community

!pip install -q langchain-groq langchain


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 6.7 MB/s eta 0:00:00


In [30]:
from langchain_core.messages import HumanMessage
from langchain_groq import ChatGroq
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

import os
os.environ["GROQ_API_KEY"] = ""  # Replace with your actual API key



In [31]:
llm = ChatGroq(
    model_name="mixtral-8x7b-32768",  # You can also use "llama2-70b-4096"
    temperature=0.7,
    max_tokens=1024
)

In [11]:
!pip install chromadb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.4 MB/s eta 0:00:00


In [12]:
import chromadb

In [14]:
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import Chroma

In [15]:

embedding_function = SentenceTransformerEmbeddings(model_name= "intfloat/e5-large")



<ipython-input-15-7537ac681c83>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name= "intfloat/e5-large")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datas

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [16]:
from langchain_community.vectorstores import Chroma

from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import TextLoader


In [17]:


from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator = '.',
    chunk_size = 200,
)


In [18]:

texts = text_splitter.create_documents([ans])
persist_directory = "/content/drive/MyDrive/testing/db_test"
vectordb = Chroma.from_documents(documents=texts, embedding=embedding_function, persist_directory=persist_directory)

In [19]:


vectordb = Chroma(
    persist_directory="/content/drive/MyDrive/testing/db_test",
    embedding_function=embedding_function
)

<ipython-input-19-eac6614780a5>:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(


In [20]:
len(texts)

2

In [25]:

# Now you can perform queries
query = "How many countries participated in the exercise"

# Method 1: Basic similarity search
docs = vectordb.similarity_search(query,k=1)

In [26]:
len(docs)

1

In [35]:
def retrieve(CONTEXT,query):
  template=prompt = f"""
  use the following CONTEXT to answer the QUESTION at the end.
  If you don't know the answer, just say that I don't know, don't try to make up an answer.
  Also remember dont use any external knowledge and only refer to this CONTEXT to answer question.
  Consider this CONTEXT as the ultimate truth.

  CONTEXT: {CONTEXT}
  QUESTION: {query}

  """
  prompt = PromptTemplate(
      input_variables=["query","CONTEXT"],
      template=template
  )

  chain = LLMChain(llm=llm, prompt=prompt)
  response = chain.invoke({"query": query, "CONTEXT":CONTEXT})
  return response['text']





In [40]:
CONTEXT = docs[0].page_content
answer = retrieve(CONTEXT,query)


In [41]:
answer

'Based on the information provided in the context, nine nations participated in the naval exercise. This includes India and eight other countries.'